# How much do NWS forecast zones overlay with counties

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
import numpy as np
import requests
from pathlib import Path

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 10000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

In [6]:
pd.options.display.float_format = "{:.2f}".format

---

# Lookup table for NWS zones to home counties

#### Read [zone-county correlation file](https://www.weather.gov/gis/ZoneCounty)

In [7]:
zones = pd.read_csv(
    "data/raw/nws_zone_to_county_lookup.txt",
    sep="|",
    header=0,
    names=[
        "state",
        "zone",
        "cwa",
        "name",
        "state_zone",
        "county",
        "fips",
        "time_zone",
        "fe_area",
        "lat",
        "lon",
    ],
)

In [8]:
zones.head()

,state,zone,cwa,name,state_zone,county,fips,time_zone,fe_area,lat,lon
0,NM,201,ABQ,Northwest Plateau,NM201,San Juan,35045,M,nw,36.43,-108.41
1,NM,202,ABQ,Chuska Mountains,NM202,McKinley,35031,M,nw,36.04,-108.93
2,NM,202,ABQ,Chuska Mountains,NM202,San Juan,35045,M,nw,36.04,-108.93
3,NM,203,ABQ,Far Northwest Highlands,NM203,Rio Arriba,35039,M,nc,36.66,-107.10
4,NM,203,ABQ,Far Northwest Highlands,NM203,Sandoval,35043,M,nc,36.66,-107.10


---

## Get NWS Zones

In [9]:
zones_gdf = gpd.read_file("data/raw/public_forecast_zones/z_07se22.shp")

In [10]:
zones_gdf.columns = zones_gdf.columns.str.lower()

In [11]:
zones_gdf = zones_gdf.to_crs("esri:102003")

In [12]:
zones_gdf["zone_area_sqm"] = (zones_gdf.area / 2.788e7).round(2)

---

## Get counties

In [13]:
counties_src = gpd.read_file("data/raw/usa_counties_esri_simple_mainland.geojson")

In [14]:
counties_src[counties_src["name"] == "Los Angeles County"]

,fid,name,state_name,state_fips,cnty_fips,fips,population,pop_sqmi,pop2010,pop10_sqmi,white,black,ameri_es,asian,hawn_pi,hispanic,other,mult_race,males,females,age_under5,age_5_9,age_10_14,age_15_19,age_20_24,age_25_34,age_35_44,age_45_54,age_55_64,age_65_74,age_75_84,age_85_up,med_age,med_age_m,med_age_f,households,ave_hh_sz,hsehld_1_m,hsehld_1_f,marhh_chd,marhh_no_c,mhh_child,fhh_child,families,ave_fam_sz,hse_units,vacant,owner_occ,renter_occ,no_farms12,ave_size12,crop_acr12,ave_sale12,sqmi,no_farms17,ave_size17,crop_acr17,ave_sale17,shape_leng,shape_length,shape_area,geometry
175,204,Los Angeles County,California,06,037,06037,10173432,2490.40,9818605,2402.30,4936599,856874,72828,1346865,26094,4687889,2140632,438713,4839654,4978951,645793,633690,678845,753630,752788,1475731,1430326,1368947,1013156,568470,345603,151626,34.80,33.60,35.90,3241204,2.98,360530,424398,790374,690291,115984,296976,2194080,3.58,3445076,203872,1544749,1696455,1294,71,59556,149225,4085.07,1035,56,29624,149380,8.83,8.83,1.04,"MULTIPOLYGON (((-118.59513 33.03596, -118.5967..."


In [15]:
counties_gdf = counties_src[
    [
        "name",
        "state_name",
        "state_fips",
        "cnty_fips",
        "fips",
        "population",
        "shape_area",
        "geometry",
    ]
].copy()

In [16]:
counties_gdf = counties_gdf.to_crs("esri:102003")

In [17]:
counties_gdf["county_area"] = (counties_gdf.area / 2.59e6).round(2)

In [18]:
counties_gdf[counties_gdf["name"] == "Harris County"]

,name,state_name,state_fips,cnty_fips,fips,population,shape_area,geometry,county_area
429,Harris County,Georgia,13,145,13145,35981,0.12,"POLYGON ((1033693.784 -457074.285, 1033736.345...",473.01
2589,Harris County,Texas,48,201,48201,4778365,0.42,"MULTIPOLYGON (((89170.066 -887556.840, 89098.6...",1748.81


---

## Apportion the zones

In [19]:
poly_intersections = zones_gdf.overlay(
    counties_gdf, how="intersection", keep_geom_type=False
)

In [20]:
poly_intersections["area"] = (poly_intersections.area / 2.59e6).round(2)

In [21]:
poly_intersections["area_share"] = (
    (poly_intersections["area"] / poly_intersections["county_area"]) * 100
).round(4)

In [22]:
poly_intersections["pop_share"] = (
    ((poly_intersections["area_share"] / 100) * poly_intersections["population"])
).round(0)

In [23]:
poly_intersections[poly_intersections["name_2"] == "Los Angeles County"][
    ["state_zone", "name_1", "name_2", "pop_share"]
]

,state_zone,name_1,name_2,pop_share
21254,CA055,San Bernardino County Mountains,Los Angeles County,51.00
21255,CA060,Apple and Lucerne Valleys,Los Angeles County,244.00
21256,CA552,Orange County Coastal,Los Angeles County,122.00
21257,CA554,Orange County Inland,Los Angeles County,397.00
21258,CA048,San Bernardino and Riverside County Valleys-Th...,Los Angeles County,519.00
21259,CA335,Grapevine,Los Angeles County,51.00
21260,CA336,Frazier Mountain Communities,Los Angeles County,0.00
21261,CA338,Mojave Desert Slopes,Los Angeles County,102.00
21262,CA339,Mojave Desert,Los Angeles County,2045.00
21263,CA059,Antelope Valley,Los Angeles County,2711148.00


---

## Export

In [24]:
poly_intersections.to_file(
    "data/processed/public_forecast_zones_pop.geojson", driver="GeoJSON"
)